In [1]:
import polars as pl
from pathlib import Path

# Proje yolları
RAW_DATA_PATH = Path("../data/raw/MetroPT3(AirCompressor).csv")
PROCESSED_DATA_PATH = Path("../data/processed/metropt3_optimized.parquet")

# Lazy okuma — RAM'e tam yüklemez, sadece şemayı biraz okur kabaca bilgileri tahmin eder
df_lazy = pl.scan_csv(RAW_DATA_PATH)

# Şemayı gör — sütun isimleri ve mevcut veri tipleri
print(df_lazy.schema)

Schema({'': Int64, 'timestamp': String, 'TP2': Float64, 'TP3': Float64, 'H1': Float64, 'DV_pressure': Float64, 'Reservoirs': Float64, 'Oil_temperature': Float64, 'Motor_current': Float64, 'COMP': Float64, 'DV_eletric': Float64, 'Towers': Float64, 'MPG': Float64, 'LPS': Float64, 'Pressure_switch': Float64, 'Oil_level': Float64, 'Caudal_impulses': Float64})


/var/folders/mw/y5zpf1vn17bgfjkddgx020p00000gn/T/ipykernel_92280/3953783389.py:12: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  print(df_lazy.schema)
